In [ ]:
pip install tensorflow

In [8]:
from tensorflow import keras

# 옵티마이저 설정
# optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)
optimizer = keras.optimizers.SGD(learning_rate=0.01, decay=1e-4)

# 모델 생성
model = keras.Sequential([
    # 여기에 모델 레이어를 추가하세요
])

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 훈련
# model.fit(train_images, train_labels, epochs=5, batch_size=32)


In [10]:
def exponential_decay(lr0, s): 
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 ** (epoch / s)
    return exponential_decay_fn
exponential_decay_fn = exponential_decay(lr0=0.01, s=20)


In [11]:
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else: return 0.001
lr_scheduler = keras.callbacks.LearningRateScheduler(piecewise_constant_fn)

1. y=-0.1x^4+2x^2+x+2 의 최대값을 제시된 방법으로 구하시오

1) 미분법을 이용

In [3]:
from scipy.optimize import minimize_scalar
import numpy as np

# 4차 함수 정의
def f(x):
    return -0.1 * x**4 + 2 * x**2 + x + 2

# 함수 f(x)를 최대화하는 x를 찾는 함수 정의
def find_max():
    # 함수 f(x)의 음수를 최소화하는 x를 찾음 (최대값을 찾기 위해)
    result = minimize_scalar(lambda x: -f(x), method='bounded', bounds=(-10, 10))
    if result.success:
        return result.x, f(result.x)  # 최대값을 가진 x와 해당 위치에서의 함수 값 반환
    else:
        return None, None  # 해를 찾을 수 없는 경우 None 반환

# 최대값 찾기
max_x, max_val = find_max()

# 결과 출력
if max_x is not None:
    print("최대값을 가지는 x:", max_x)
    print("최대값:", max_val)
else:
    print("최대값을 찾을 수 없습니다.")


최대값을 가지는 x: 3.2805585009159755
최대값: 15.222484335028529


2) 뉴톤법을 이용 초기값 (x=-5)

In [5]:
import sympy as sp

# 초기값 설정
x = -5

# 함수 정의
f = -0.1 * sp.Symbol('x')**4 + 2 * sp.Symbol('x')**2 + sp.Symbol('x') + 2

# 함수 f(x)를 미분하여 f'(x)를 구함
f_prime = sp.diff(f, sp.Symbol('x'))

# 뉴턴-랩슨 반복
for _ in range(10):  # 일정 횟수 반복
    # 현재 위치에서의 함수값과 미분값 계산
    fx = f.subs(sp.Symbol('x'), x)
    fx_prime = f_prime.subs(sp.Symbol('x'), x)
    
    # 다음 위치 계산
    x = x - fx / fx_prime
    
# 최대값에서의 x값과 최대값 계산
max_x = x
max_val = f.subs(sp.Symbol('x'), max_x)

# 결과 출력
print("뉴턴-랩슨 방법을 이용한 최대값의 x:", max_x)
print("최대값:", max_val)


뉴턴-랩슨 방법을 이용한 최대값의 x: -4.33096985608591
최대값: 0


3) 경사하강법 (초기값 x=-5, 학습율 : 최대값 찾을때까지)

In [7]:
import sympy as sp

# 초기값 설정
x = -5
learning_rate = 0.01  # 학습율

# 함수 정의
f = -0.1 * sp.Symbol('x')**4 + 2 * sp.Symbol('x')**2 + sp.Symbol('x') + 2

# 함수를 최대화하는 방향으로 경사 상승법 반복
for _ in range(1000):  # 일정 횟수 반복 또는 수렴할 때까지
    # 현재 위치에서의 함수값과 그래디언트 계산
    fx = f.subs(sp.Symbol('x'), x)
    gradient = sp.diff(f, sp.Symbol('x')).subs(sp.Symbol('x'), x)
    
    # 경사 상승법 업데이트
    x = x + learning_rate * gradient
    
# 최대값 계산
max_val = f.subs(sp.Symbol('x'), x)

# 결과 출력
print("경사 하강법을 이용한 최대값의 x:", x)
print("최대값:", max_val)


경사 하강법을 이용한 최대값의 x: -3.02896613891181
최대값: 8.90291126649044


4) 유전알고리즘

In [6]:
import numpy as np
import random

# 주어진 함수 정의
def objective_function(x):
    return -0.1*x**4 + 2*x**2 + x + 2

# 후보해 생성
def initialize_population(num_candidates):
    population = []
    for _ in range(num_candidates):
        candidate = np.random.uniform(-10, 10)  # 임의의 범위에서 후보해 생성
        population.append(candidate)
    return population

# 선택 연산
def selection(population):
    fitness_scores = [objective_function(candidate) for candidate in population]
    max_fitness_index = np.argmax(fitness_scores)
    return population[max_fitness_index]

# 교차 연산
def crossover(parent1, parent2):
    crossover_point = random.uniform(0, 1)  # 교차점 비율을 0에서 1 사이에서 무작위로 선택
    child = crossover_point * parent1 + (1 - crossover_point) * parent2  # 1점 교차
    return child

# 돌연변이 연산
def mutation(candidate):
    mutation_point = random.uniform(0, 1)  # 돌연변이 발생할 위치 선택
    mutation_amount = random.uniform(-1, 1)  # 돌연변이 양 선택
    candidate += mutation_amount * mutation_point
    return candidate

# 유전 알고리즘 실행
def genetic_algorithm(num_generations):
    num_candidates = 4
    population = initialize_population(num_candidates)
    
    for generation in range(num_generations):
        parent1 = selection(population)
        parent2 = selection(population)
        child = crossover(parent1, parent2)
        
        if random.random() < 0.1:  # 돌연변이 확률 0.1
            child = mutation(child)
        
        population.append(child)
    
    best_solution = selection(population)
    return best_solution

# 유전 알고리즘 실행 및 결과 출력
best_solution = genetic_algorithm(num_generations=100)
print("최대값:", objective_function(best_solution))
print("해:", best_solution)


최대값: 15.221518475964455
해: 3.2952474741190385


3. 2개의 통신사

In [58]:
import numpy as np

# 전이 확률 행렬
transition_matrix = np.array([[0.6, 0.4], [0.2, 0.8]])

# 현재 시점에서 각 통신사에 가입한 상태 벡터
current_state_A = np.array([1, 0])  # A통신사에 가입
current_state_B = np.array([0, 1])  # B통신사에 가입

def predict_next_state(current_state, transition_matrix):
    # 다음 시점에서 각 통신사에 가입할 확률 계산
    next_state = np.dot(current_state, transition_matrix)
    return next_state

# 예측을 3번 반복
for i in range(3):
    # A통신사에 가입한 상태에서 다음 시점에서 각 통신사에 가입할 확률
    next_state_prob_A = predict_next_state(current_state_A, transition_matrix)
    print(f"{i+1}번째 예측 - 다음 시점에서 A통신사에 가입할 확률:", next_state_prob_A)
    current_state_A = next_state_prob_A
    
for i in range(3):

# B통신사에 가입한 상태에서 다음 시점에서 각 통신사에 가입할 확률
    next_state_prob_B = predict_next_state(current_state_B, transition_matrix)
    print(f"{i+1}번째 예측 - 다음 시점에서 B통신사에 가입할 확률:", next_state_prob_B)

    # 다음 시점 예측을 현재 상태로 업데이트
    
    current_state_B = next_state_prob_B


1번째 예측 - 다음 시점에서 A통신사에 가입할 확률: [0.6 0.4]
2번째 예측 - 다음 시점에서 A통신사에 가입할 확률: [0.44 0.56]
3번째 예측 - 다음 시점에서 A통신사에 가입할 확률: [0.376 0.624]
1번째 예측 - 다음 시점에서 B통신사에 가입할 확률: [0.2 0.8]
2번째 예측 - 다음 시점에서 B통신사에 가입할 확률: [0.28 0.72]
3번째 예측 - 다음 시점에서 B통신사에 가입할 확률: [0.312 0.688]


In [59]:
import numpy as np

# 전이 확률 행렬
transition_matrix = np.array([[0.6, 0.4],
                               [0.2, 0.8]])

# 전이 확률 행렬의 전치행렬
transition_matrix_transpose = transition_matrix.T

# 고유값과 고유벡터 계산
eigenvalues, eigenvectors = np.linalg.eig(transition_matrix_transpose)

# 안정 상태 분포는 고유값이 1인 고유벡터
stable_distribution = eigenvectors[:, np.argmax(np.abs(eigenvalues))].real
stable_distribution /= stable_distribution.sum()

print("A통신사의 장기 시장점유율:", stable_distribution[0])
print("B통신사의 장기 시장점유율:", stable_distribution[1])


A통신사의 장기 시장점유율: 0.3333333333333333
B통신사의 장기 시장점유율: 0.6666666666666667
